# 📘 S1W4D1：数组算法与张量维度（Arrays & Tensor Dimensions）

## 1. 核心概念：内存中的“线性世界”

**底层练习（The Connection）：**
无论是`List`还是`Tensor`，在计算机物理内存中都是**连续的一维长条**.

- **LeetCode**：通常教你如何在这个一维长条上高校移动（双指针、滑动窗口）
- **PyTroch**：`view`和`reshape`并不改变物理内存里面的数据顺序，只是改变了我们“看”它的步长（**Stride**）和视角。

## 2. PyTroch维度实现（The "Job" Skills）

我们先解决工程中最头疼的维度报错问题。

### 2.1 View vs Reshape （经典面试题）

In [2]:
import torch

# 先创建一个简单的2x3张量

t = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])
print(f"原始形状{t.shape}")

# 1. view：零拷贝，纯试图变化
# 要求：Tensor在内存中必须是连续的(Contiguous)
t_view = t.view(6)
print(f"View的结果：{t_view}")

# 2. Permute（维度交换）会破坏连续性
t_permuted = t.permute(1,0) # 变成3x2
print(f"Permuted维度交换后的形状{t_permuted.shape}")

# 尝试直接view会报错
try:
    t_permuted.view(6)
except RuntimeError as e:
    print(f"\n[预期报错] View erroe:{e}")

# 3. Reshape：更加鲁棒
# 如果内存不连续，它会自动复制 一份数据（Clone）再变型，不会报错
t_reshape = t_permuted.reshape(6)
print(f"Reshape result (success): {t_reshape}")

# 4. 最佳实践：先contiguous()再view()
t_safe = t_permuted.contiguous().view(6)
print(f"Safe View result: {t_safe}")

原始形状torch.Size([2, 3])
View的结果：tensor([1, 2, 3, 4, 5, 6])
Permuted维度交换后的形状torch.Size([3, 2])

[预期报错] View erroe:view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.
Reshape result (success): tensor([1, 4, 2, 5, 3, 6])
Safe View result: tensor([1, 4, 2, 5, 3, 6])


> **💡 面试考点**: 为什么 `permute` 后不能直接 `view`？因为 `permute` 只是改变了索引的步长，内存里的数据还是按原来的顺序排的。`view` 假设数据是按新形状顺序排列的，所以会冲突。

### 2.2 Unsqueeze & Squeeze (推理必备)

在 Week 3 的 `predict.py` 中用到了 `unsqueeze`，这里复习一下本质。

In [3]:
# 模拟一张MNIST图片：[C, H, W]
img = torch.randn(1, 28, 28)

# 错误示范：直接送入模型 (模型期待 [Batch, C, H, W])
# model(img) -> 会报错

# 正确错发：增加Batch维度
img_batch = img.unsqueeze(0)  # 在第0维增加
print(f"Unsqueeze shape:{img_batch.shape}") # [1, 1, 28, 28]

# 还原：压缩维度 (去除维度为 1 的轴)
img_squeezed = img_batch.squeeze()
print(f"Squeeze shape: {img_squeezed.shape}") # [28, 28] (注意：C=1 也被压掉了，小心使用！)

# 安全还原：指定压掉第 0 维
img_safe_squeeze = img_batch.squeeze(0)
print(f"Safe squeeze shape: {img_safe_squeeze.shape}") # [1, 28, 28]

Unsqueeze shape:torch.Size([1, 1, 28, 28])
Squeeze shape: torch.Size([28, 28])
Safe squeeze shape: torch.Size([1, 28, 28])


## 3 LeetCode 算法突击 (The "Interview" Skills)

### 3.1 移动零 (Move Zeroes) - Easy

  * **LeetCode 链接**: [283. Move Zeroes](../../LeetCode%20practice/251-300.ipynb)
  * **AI 关联**: 类似于 PyTorch 中的 **Masking (掩码)** 操作或 **Sparse Tensor (稀疏张量)** 的概念。
  * **核心思路**: **双指针 (Two Pointers)**。一个指针指向“非零元素应该放的位置”，另一个指针遍历数组。

### 3.2 盛最多水的容器 (Container With Most Water) - Medium

  * **LeetCode 链接**: [11. Container With Most Water](../../LeetCode%20practice/1-50.ipynb)
  * **AI 关联**: 训练思维。我们需要找到一个“最优解”（最大面积），通过不断调整边界（左右指针）来逼近全局最优，有点像梯度下降寻找 Loss 最低点的简化版逻辑。
  * **核心思路**: **双指针向内收敛**。永远移动“较短”的那块板，因为移动长板只可能导致面积变小（宽度变小，高度受限于短板）。

### 3.3 子数组最大平均数 I (Maximum Average Subarray I) - Easy

  * **LeetCode 链接**: [643. Maximum Average Subarray I](../../LeetCode%20practice/601-650.ipynb)
  * **AI 关联**: **极度重要！这就是卷积 (Convolution) 的一维版本。**
  * **核心思路**: **滑动窗口 (Sliding Window)**。卷积核在图像上滑动，本质上就是滑动窗口在数组上滑动。不要重复计算重叠部分。

## 4 总结与作业

### 4.1 今日复盘

1.  **维度变换**: 记住了 `permute` 后接 `view` 必须先用 `.contiguous()`。
2.  **算法逻辑**:
      * 双指针用于压缩空间或寻找两端边界。
      * 滑动窗口用于处理连续子序列（卷积的雏形）。

### 4.2 课后作业

1.  在 Notebook 中运行以上所有代码。
2.  **挑战任务**: 结合今天学的，手动实现一个简单的 **"1D 卷积"**。
      * 输入: 一个长为 10 的 PyTorch Tensor。
      * 卷积核: 一个长为 3 的 Tensor (例如 `[0.5, 1.0, 0.5]`)。
      * 任务: 利用循环或滑动窗口思路，计算输出 Tensor。
      * *(不做强制要求，但做了会对 CNN 理解极深)*

In [3]:
import torch
import torch.nn.functional as F

# 1. 模拟输入数据(比如 10 天的股价，或者一张图片的某一行像素)
# 长度为 10
input_data = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0])

# 2. 定义卷积核（kernel）
# 这是一个“平滑”滤波器：中间权重高1.0，两边权重低0.5
# 长度为3
kernel = torch.tensor([0.5, 1.0, 0.5])
print(f"输入数据:{input_data}")
print(f"卷积核:{kernel}")

输入数据:tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])
卷积核:tensor([0.5000, 1.0000, 0.5000])


#### 手动实现卷积

In [4]:
output_len = len(input_data) - len(kernel) + 1
output_manual = []

print("\n--- 开始手动卷积 ---")

# 滑动窗口循环
for i in range(output_len):
    # 1. 切片: 取出当前的窗口[i: i + 3]
    window = input_data[i : i + len(kernel)]

    # 2. 对应元素相乘 (Element-wise multiplication)
    # 比如: [1, 2, 3] * [0.5, 1.0, 0.5] = [0.5, 2.0, 1.5]
    weighted_window = window * kernel
    # 3. 求和 (Sum)
    # 0.5 + 2.0 + 1.5 = 4.0
    result = torch.sum(weighted_window)

    output_manual.append(result.item())

    # 打印前两步的过程帮助理解
    if i < 2: 
        print(f"窗口 {i}: 数据{window.tolist()} * 核{kernel.tolist()} -> 求和 = {result.item()}")

output_manual_tensor = torch.tensor(output_manual)
print(f"\n手动计算结果: {output_manual_tensor}")


--- 开始手动卷积 ---
窗口 0: 数据[1.0, 2.0, 3.0] * 核[0.5, 1.0, 0.5] -> 求和 = 4.0
窗口 1: 数据[2.0, 3.0, 4.0] * 核[0.5, 1.0, 0.5] -> 求和 = 6.0

手动计算结果: tensor([ 4.,  6.,  8., 10., 12., 14., 16., 18.])


#### 用 PyTorch 官方 API 验证

In [5]:
print("\n--- PyTorch 官方 API 验证 ---")

# 1. 调整输入维度
# 原始 input_data 是 [10]
# 我们需要把它变成 [1, 1, 10] (1个Batch, 1个通道, 长度10)
input_reshaped = input_data.unsqueeze(0).unsqueeze(0)

# 2. 调整卷积核维度
# 原始 kernel 是 [3]
# 官方要求卷积核是 [Out_Channels, In_Channels, Kernel_Size]
# 变成 [1, 1, 3]
kernel_reshaped = kernel.unsqueeze(0).unsqueeze(0)

print(f"调整后的输入维度: {input_reshaped.shape}")
print(f"调整后的卷积核维度: {kernel_reshaped.shape}")

# 3. 调用官方卷积函数
output_pytorch = F.conv1d(input_reshaped, kernel_reshaped)

# 4. 把结果降维回来看一眼
output_pytorch_squeezed = output_pytorch.squeeze()
print(f"官方计算结果: {output_pytorch_squeezed}")

# 5. 对比两者是否一致
is_same = torch.allclose(output_manual_tensor, output_pytorch_squeezed)
print(f"\n结果一致吗? {'✅ 是的!' if is_same else '❌ 不一致'}")


--- PyTorch 官方 API 验证 ---
调整后的输入维度: torch.Size([1, 1, 10])
调整后的卷积核维度: torch.Size([1, 1, 3])
官方计算结果: tensor([ 4.,  6.,  8., 10., 12., 14., 16., 18.])

结果一致吗? ✅ 是的!
